In [1]:
seed=40
embedding_dim = 100
vocab_size = 12112
lower = True
FN0 = 'train_data'
FN = 'vocabulary-embedding'
n_fold = 5

In [2]:
import _pickle as pickle
with open('data/5_fold_input.pkl', 'rb') as contentFile:
    (X, Y) = pickle.load(contentFile)

In [3]:
for i in range(len(X)):
    print(len(X[i]), len(Y[i])) 

320 320
320 320
320 320
320 320
320 320


In [4]:
if lower:
    content = [h.lower() for t in X for y in t for h in y]

In [5]:
print(len(X))
print(len(X[0]))

5
320


In [6]:
from collections import Counter
from itertools import chain
def get_vocab(lst):
    vocabcount = Counter(w for txt in lst for w in txt)
    vocab = list(map(lambda x: x[0], sorted(vocabcount.items(), key=lambda x: -x[1])))
    return vocab, vocabcount

In [7]:
all_X = list(X[0])
for i in range(1, len(X)):
    all_X+=list(((X[i])))
print(len(all_X))

1600


In [8]:
import numpy as np
print(np.array(X).shape)
print(np.array(Y).shape)
print(len(X[0]))

(5, 320)
(5, 320)
320


In [9]:
vocab, vocabcount = get_vocab(all_X)

In [10]:
print (vocab[:50])
print ('...',len(vocab))

['.', 'the', ',', 'and', 'to', 'I', 'a', 'was', 'in', 'of', 'The', 'for', 'hotel', 'room', 'at', 'it', 'is', 'we', 'that', 'my', 'were', 'with', 'had', 'not', '!', 'on', 'Chicago', 'this', 'but', 'stay', 'very', 'our', 'have', 'We', "n't", 'you', 'they', 'be', 'would', 'there', 'from', 'as', 'are', 'all', 'me', 'staff', 'great', 'service', 'so', 'did']
... 12110


In [11]:
i = 0
for key in vocab:
    print(key, vocabcount[key])
    if(i > 10):
        break
    i += 1

. 13678
the 12806
, 8715
and 7802
to 6702
I 6357
a 6331
was 5986
in 3784
of 3287
The 3145
for 2786


In [12]:

"""with open('data/vocabList.pkl', 'w+b') as output_file:
    pickle.dump(vocab, output_file)
with open('data/vocab_counter.pkl', 'w+b') as output_file:
    pickle.dump(vocabcount, output_file)
print('finished')"""

"with open('data/vocabList.pkl', 'w+b') as output_file:\n    pickle.dump(vocab, output_file)\nwith open('data/vocab_counter.pkl', 'w+b') as output_file:\n    pickle.dump(vocabcount, output_file)\nprint('finished')"

In [13]:
print('here')

here


In [14]:
empty = 0 # RNN mask of no data
eos = 1  # end of sentence
start_idx = eos+1 # first real word

In [15]:
def get_idx(vocab, vocabcount):
    word2idx = dict((word, idx+start_idx) for idx,word in enumerate(vocab))
    word2idx['<empty>'] = empty
    word2idx['<eos>'] = eos
    
    idx2word = dict((idx,word) for word,idx in word2idx.items())

    return word2idx, idx2word

In [16]:
word2idx, idx2word = get_idx(vocab, vocabcount)

In [17]:
"""with open('data/word2idx.pkl', 'w+b') as output_file:
    pickle.dump(word2idx, output_file)
with open('data/idx2word.pkl', 'w+b') as output_file:
    pickle.dump(idx2word, output_file)
print('finished')"""

"with open('data/word2idx.pkl', 'w+b') as output_file:\n    pickle.dump(word2idx, output_file)\nwith open('data/idx2word.pkl', 'w+b') as output_file:\n    pickle.dump(idx2word, output_file)\nprint('finished')"

In [18]:
import os
cwd = os.getcwd()

In [19]:
fname = '/data/glove.6b/glove.6B.%dd.txt'%embedding_dim
fname = cwd + fname

In [20]:
glove_n_symbols = !wc -l {fname}

In [21]:
glove_n_symbols = int(glove_n_symbols[0].split()[0])
glove_n_symbols

400000

In [22]:
import numpy as np
glove_index_dict = {}
glove_embedding_weights = np.empty((glove_n_symbols, embedding_dim))
globale_scale=.1
with open(fname, 'r') as fp:
    i = 0
    for l in fp:
        l = l.strip().split()
        w = l[0]
        glove_index_dict[w] = i
        glove_embedding_weights[i,:] = np.array(list(map(float,l[1:])))
        i += 1
glove_embedding_weights *= globale_scale

In [23]:
glove_embedding_weights.std()

0.040815727600190289

In [24]:
for w,i in glove_index_dict.items():
    w = w.lower()
    if w not in glove_index_dict:
        glove_index_dict[w] = i

In [25]:
# generate random embedding with same scale as glove
np.random.seed(seed)
shape = (vocab_size, embedding_dim)
scale = glove_embedding_weights.std()*np.sqrt(12)/2 # uniform and not normal
embedding = np.random.uniform(low=-scale, high=scale, size=shape)
print ('random-embedding/glove scale', scale, 'std', embedding.std())

# copy from glove weights of words that appear in our short vocabulary (idx2word)
c = 0
for i in range(vocab_size):
    w = idx2word[i]
    g = glove_index_dict.get(w, glove_index_dict.get(w.lower()))
    if g is None and w.startswith('#'): # glove has no hastags (I think...)
        w = w[1:]
        g = glove_index_dict.get(w, glove_index_dict.get(w.lower()))
    if g is not None:
        embedding[i,:] = glove_embedding_weights[g,:]
        c+=1
print ('number of tokens, in small vocab, found in glove and copied to embedding', c,c/float(vocab_size))

random-embedding/glove scale 0.0706949139514 std 0.0408301773961
number of tokens, in small vocab, found in glove and copied to embedding 10752 0.8877146631439894


In [26]:
embedding_size = embedding.shape

In [27]:
glove_thr = 0.5

In [28]:
word2glove = {}
for w in word2idx:
    if w in glove_index_dict:
        g = w
    elif w.lower() in glove_index_dict:
        g = w.lower()
    elif w.startswith('#') and w[1:] in glove_index_dict:
        g = w[1:]
    elif w.startswith('#') and w[1:].lower() in glove_index_dict:
        g = w[1:].lower()
    else:
        continue
    word2glove[w] = g

In [29]:
normed_embedding = embedding/np.array([np.sqrt(np.dot(gweight,gweight)) for gweight in embedding])[:,None]

nb_unknown_words = 100

glove_match = []
for w,idx in word2idx.items():
    if idx >= vocab_size-nb_unknown_words and w.isalpha() and w in word2glove:
        gidx = glove_index_dict[word2glove[w]]
        gweight = glove_embedding_weights[gidx,:].copy()
        # find row in embedding that has the highest cos score with gweight
        gweight /= np.sqrt(np.dot(gweight,gweight))
        score = np.dot(normed_embedding[:vocab_size-nb_unknown_words], gweight)
        while True:
            embedding_idx = score.argmax()
            s = score[embedding_idx]
            if s < glove_thr:
                break
            if idx2word[embedding_idx] in word2glove :
                glove_match.append((w, embedding_idx, s)) 
                break
            score[embedding_idx] = -1
glove_match.sort(key = lambda x: -x[2])
print ('# of glove substitutes found', len(glove_match))

# of glove substitutes found 67


In [30]:
for orig, sub, score in glove_match[-20:-10]:
    print (score, orig,'=>', idx2word[sub])

0.650219937579 Reed => Matthews
0.646316348902 curb => prevent
0.642169179713 hasnt => isnt
0.640902690131 marveling => amazement
0.639570344528 insults => shouting
0.619533255341 gratus => 3114
0.615517664155 torture => execution
0.614677928838 hides => hide
0.594263116271 freely => directly
0.594157339553 puddle => mud


In [31]:
glove_idx2idx = dict((word2idx[w],embedding_idx) for  w, embedding_idx, _ in glove_match)

In [32]:
X = [[[word2idx[token] for token in content] for content in fold] for fold in X]
len(X)

5

In [33]:
np.array(X).shape

(5, 320)

In [34]:
import _pickle as pickle
with open('data/%s.pkl'%FN,'wb') as fp:
    pickle.dump((embedding, idx2word, word2idx, glove_idx2idx),fp,-1)

In [35]:
import _pickle as pickle
with open('data/%s.data.pkl'%FN,'wb') as fp:
    pickle.dump((X,Y),fp,-1)

In [36]:
print(np.array(X).shape)
print(np.array(Y).shape)

(5, 320)
(5, 320)


In [37]:
"""from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=320, random_state=seed)"""
import itertools
X_train = [[]]*len(X)
X_test = [[]]*len(X)
Y_train = [[]]*len(X)
Y_test = [[]]*len(X)
for i in range(len(X)):
    X_train[i] = list(itertools.chain.from_iterable(X[0:i] + X[i+1:]))
    Y_train[i] = list(itertools.chain.from_iterable(Y[0:i] + Y[i+1:]))
    X_test[i] = (list(X[i]))
    Y_test[i] = (list(Y[i]))

In [38]:
len(X_train), len(Y_train), len(X_test), len(Y_test)

(5, 5, 5, 5)

In [39]:
np.array(X_train).shape, np.array(Y_train).shape, np.array(X_test).shape, np.array(Y_test).shape

((5, 1280), (5, 1280), (5, 320), (5, 320))

In [40]:
print(X_train[0][0])

[35, 57, 161, 187, 150, 2461, 1252, 10, 108, 9, 144, 8, 10006, 1041, 30, 5716, 8, 2095, 839, 23, 8, 580, 16, 55, 492, 2, 1944, 736, 542, 4, 577, 20, 1926, 471, 11, 4037, 3, 549, 4, 8, 413, 20, 8004, 58, 160, 3, 1192, 11, 3, 15, 5, 8, 580, 20, 5628, 60, 3, 3969, 2355, 2, 3815, 3815, 107, 2, 4931, 7116, 14, 786, 10, 3, 138, 2]


In [41]:
import keras
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences
maxlen = 300
trainX = [[]]*n_fold
testX = [[]]*n_fold
trainY = [[]]*n_fold
testY = [[]]*n_fold
for i in range(n_fold):
    trainX[i] = pad_sequences(X_train[i], maxlen=maxlen, value=0.)
    testX[i] = pad_sequences(X_test[i], maxlen=maxlen, value=0.)

    trainY[i] = keras.utils.to_categorical(Y_train[i], num_classes=2)
    testY[i] = keras.utils.to_categorical(Y_test[i], num_classes=2)


Using TensorFlow backend.


In [42]:
np.array(trainX).shape, np.array(trainY).shape, np.array(testX).shape, np.array(testY).shape

((5, 1280, 300), (5, 1280, 2), (5, 320, 300), (5, 320, 2))

In [43]:
#embedding = tf.cast(embedding, tf.float32)

In [44]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, RepeatVector, Flatten, Lambda
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D
from keras.regularizers import l2
from keras.layers import SpatialDropout1D
import random
import keras.backend as K
from keras import layers
import random

In [45]:
random.seed(seed)
np.random.seed(seed)
p_W, p_U, p_dense, p_emb, weight_decay = 0.2, 0.2, 0, 0.1, 0.0
optimizer = 'adam'
LR = 1e-4
batch_size=64
rnn_size = 128
rnn_layers = 1

In [46]:
regularizer = l2(weight_decay) if weight_decay else None

In [47]:
print(regularizer)

None


In [48]:
"""model = None
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim,#mask_zero=False,
                    input_length=maxlen,
                    embeddings_regularizer=regularizer,
                    name='embedding_1'))
model.add(Dropout(p_emb))
model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
for i in range(rnn_layers):
    lstm = LSTM(rnn_size, return_sequences=False, # batch_norm=batch_norm,
                recurrent_regularizer=regularizer, kernel_regularizer=regularizer,
                bias_regularizer=regularizer, recurrent_dropout=p_W, dropout=p_U,
                name='lstm_%d'%(i+1)
                  )
    model.add(lstm)

model.add(Dense(2, activation='softmax'))"""

In [49]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, RepeatVector, Flatten, Lambda, Masking
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D
from keras.regularizers import l2
from keras.layers import SpatialDropout1D, Input
import random
import keras.backend as K
from keras import layers
from keras.models import Model

def create_model():
    main_input = Input(shape=(300,), dtype='float32', name='main_input')
    x = Embedding(vocab_size, embedding_dim,#mask_zero=False,
                        input_length=maxlen,
                        embeddings_regularizer=regularizer,
                        name='embedding_1')(main_input)
    x = Dropout(p_emb)(x)

#     cov1 = Conv1D(filters=128, kernel_size=1, padding='same', activation='relu')(x)
#     pool_1 = MaxPooling1D(pool_size=2)(cov1)

    cov2 = Conv1D(filters=64, kernel_size=2, padding='same', activation='relu')(x)
    pool_2 = MaxPooling1D(pool_size=2)(cov2)

#     cov3 = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(x)
    #pool_3 = MaxPooling1D(pool_size=3)(cov3)

    #x = keras.layers.concatenate([cov1, cov2, cov3])
    x = pool_2
    x = Masking(mask_value=0.)(x)

    lstm = LSTM(rnn_size, return_sequences=False, # batch_norm=batch_norm,
                    recurrent_regularizer=regularizer, kernel_regularizer=regularizer,
                    bias_regularizer=regularizer, recurrent_dropout=p_W, dropout=p_U,
                    name='lstm_%d'%(i+1)
                    )(x)

    main_output = Dense(2, activation='softmax')(lstm)
    model = Model(inputs = [main_input], outputs = [main_output])
    return model

#log 4:10 AM: added masking, return_sequences = True

In [50]:
def str_shape(x):
    return 'x'.join(map(str,x.shape))
    
def inspect_model(model):
    for i,l in enumerate(model.layers):
        print (i, 'cls=%s name=%s'%(type(l).__name__, l.name))
        weights = l.get_weights()
        for weight in weights:
            print (str_shape(weight)),
        print

In [51]:
model = create_model()

In [52]:
from keras.optimizers import Adam, RMSprop # usually I prefer Adam but article used rmsprop
# opt = Adam(lr=LR)  # keep calm and reduce learning rate
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [53]:

K.set_value(model.optimizer.lr,np.float32(LR))

In [54]:
inspect_model(model)

0 cls=InputLayer name=main_input
1 cls=Embedding name=embedding_1
12112x100
2 cls=Dropout name=dropout_2
3 cls=Conv1D name=conv1d_2
2x100x32
32
4 cls=MaxPooling1D name=max_pooling1d_2
5 cls=Masking name=masking_1
6 cls=LSTM name=lstm_1
32x512
128x512
512
7 cls=Dense name=dense_2
128x2
2


In [66]:
model.fit(trainX[3], trainY[3], validation_data = (testX[3], testY[3]), epochs = 1, verbose=1)

Train on 1280 samples, validate on 320 samples
Epoch 1/1
1280/1280 [==============================] - 17s - loss: 0.2722 - acc: 0.9273 - val_loss: 0.4302 - val_acc: 0.8625

In [68]:
result_Y = model.predict(testX[3])

In [69]:
Y_test[3][0]

True

In [70]:
testY[3][0]

array([ 0.,  1.])

In [71]:
result_Y[0]

array([ 0.04985845,  0.95014155], dtype=float32)

In [73]:
#count recalls
count_total_f = 0
count_t_neg = 0
for i in range(len(result_Y)):
    if(Y_test[3][i] == True):
        count_total_f += 1
        if(result_Y[i][0] < result_Y[i][1]):
            count_t_neg += 1
print(count_t_neg/count_total_f)
        

0.8875


In [67]:
model.evaluate(testX[3], testY[3])
#model.evaluate(trainX[3], trainY[3])

320/320 [==============================] - 0s     


[0.43021848499774934, 0.86250000000000004]

In [ ]:
#model.save('lstm_cnn')

In [327]:
from keras.models import load_model
old_model = load_model('lstm_cnn_fake_news.h5')

In [347]:
old_model.evaluate(testX[3], testY[3])
old_model.evaluate(traintX[3], traintX[3])

320/320 [==============================] - 0s     


[0.78023982048034668, 0.74062499999999998]

In [38]:
"""net = tflearn.input_data([None, maxlen])
net = tflearn.embedding(net, input_dim=vocab_size, output_dim=128)
net = tflearn.lstm(net, 128, dropout = 0.7)
net = tflearn.fully_connected(net, 2, activation='softmax')
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy')"""

In [39]:
"""len(trainX), len(trainY), len(testX), len(testY)"""

(1440, 1440, 160, 160)

In [41]:
"""model = tflearn.DNN(net, tensorboard_verbose=0, best_checkpoint_path = os.getcwd() + '/checkpoint/', 
                    best_val_accuracy=0.8)"""

In [42]:
"""model.load(os.getcwd() + '/checkpoint/5750')"""

"model.load(os.getcwd() + '/checkpoint/5750')"

In [43]:
"""model.evaluate(testX, testY)[0]"""

0.55000000000000004

In [44]:
"""numberOfRuns = 1
while(model.evaluate(testX, testY)[0]<0.89 and model.evaluate(trainX, trainY)[0]<0.99 and numberOfRuns <= 20):
    model.fit(trainX, trainY, validation_set=(testX, testY), show_metric=True, n_epoch = 50)
    print(model.evaluate(testX, testY))
    numberOfRuns+=1
    """
    

Training Step: 1609  | total loss: 0.65045 | time: 11.953s
| Adam | epoch: 070 | loss: 0.65045 - acc: 0.5617 -- iter: 1408/1440
Training Step: 1610  | total loss: 0.64975 | time: 13.541s
| Adam | epoch: 070 | loss: 0.64975 - acc: 0.5711 | val_loss: 0.76642 - val_acc: 0.4688 -- iter: 1440/1440
--


KeyboardInterrupt: 

In [ ]:
#import os
#model.save(os.getcwd())

In [ ]:
"""print('number of iterations = ' + str(numberOfRuns*50))
print('validation accuracy = ' + str(model.evaluate(testX, testY)[0]*100) + '%')"""
